In [1]:
import torch
from torch import nn
import torchvision.models as models
from PIL import Image
from torchvision import transforms

In [2]:
import sys
sys.path.append('../..')  # Navigate up to GitHub folder
from LeNet_5.classes.convolution import ConvolutionLayer

In [3]:
def to_tensor(img):
    transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    return transform(img)

In [4]:
cat_img = Image.open('../data/custom/cat108.jpg')
lion_img = Image.open('../data/custom/lion.jpg')

cat_tensor = to_tensor(cat_img)[:3,:,:].unsqueeze(0)
lion_tensor = to_tensor(lion_img)[:3,:,:].unsqueeze(0)
batch = torch.cat([cat_tensor, cat_tensor], dim=0)

In [5]:
from math import sqrt

def reshape_output(out_tensor):
    batch_size, features = out_tensor.shape
    height = width = int(sqrt(features//512))
    return out_tensor.reshape(
        batch_size, 
        512, 
        height,
        width,
        )

In [6]:
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
model.classifier = nn.Identity()
model.avgpool = nn.Identity()

out = model(batch)
out = reshape_output(out)


In [7]:
conv = ConvolutionLayer(in_channels=512, num_filters=4096, filter_size=7, efficient=True)
conv.filters[0].weight.shape

torch.Size([512, 7, 7])

In [13]:
conv = nn.Conv2d(512, 4096, 7)

In [27]:
forward = conv(out)
forward.shape

torch.Size([2, 4096, 1, 1])

In [10]:
model.classifier = nn.Identity()
model.avgpool = nn.Identity()
pred = model(cat_tensor)
labels = models.VGG16_Weights.DEFAULT.meta["categories"]
print(pred.shape)

torch.Size([1, 25088])


In [11]:
model.classifier[0].weight = nn.Parameter(model.classifier[0].weight.reshape(4096, 512, 7, 7))
print(model.classifier[0].weight.shape)

TypeError: 'Identity' object is not subscriptable

In [ ]:
class FCN_16(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = models.vgg16()

In [ ]:
def rarify_input(weights, stride, size):
    out_size = (size-1) * stride + 1
    tensor = torch.empty()